LittleGuns_01_Tonka-RBDev
Notebook created 12/15/22 to debug Little Guns and XLOptionBoard

12/14/2 - Tested and running as OptionGun03_Tonka_Prdtn
Dev version in this notebook temporarilly until I learn how to do releases.


OptionGun Structure

- HEADERS and CONSTANTS

- Broker_Converter_Van - TEMPORARY here to get Vanguard report and produce trade_log_open for BTC offers.

- GET OPTION DATA
       - Get real-time price data from Darqube.
       - Get option strings from yfinance.
       - Get ER dates from yfinance.
       - Parse option strings into common POWMax format.

12/15/22 - Added little option guns. 
- Put_STO_Gun - Screen and select PSTO offers
- Call_STO_Gun- Screen and select CSTO offers
- Put_BTC_Gun - Screen and select PBTC offers
- Call_BTC_Gun- Screen and select CBTC offers

ToDo 12/15/22 - Dev Notes
- Change best_options to best_STO everywhere.
- Use 'put' and 'call' to conform to Yahoo Finance nomencleture.
1. Convert all_options, best_options, and real-time prices to fcts. Set up calls so they can be bypassed for testing.
1. Complete end-to-end flow from each little option gun to output a csv file and into XLOptionBoard.
1. Use new column lables to split broker_report into three.
2. Delete fill-col headers. Replace with correct column headers.


In [1]:
# ****************
#
#Header and Constants - from OptionGun. This code DOES NOT RUN OptionGun.
#
# *****************
#
#   12/05/22 - Forked version for Tonka_Prdtn.
#   12/05/22 - Added tickers with high Quant rating. Tightened limits for best_options.
#   11/27/22 - Removed IEX. Now using Darqube for market data. Added many more tickers, approx. 150 total.
#
# *****************
#
# USER PARAMS
# User can change these parameters as appropriate.
#
# *****************

DAR_key = 'a5e6a13d876c4819ad02083f76ff22a3' # Get one from Darqube and enter it here, between the single quotes.

TICKERS = ('URI', 'JEF', 'BSM', 'BGSF', 'CPA', 'STLD', 'AMR', 'UNVR', 'CVCO', 'HUBG', 'ACLS', 'ASRT', \
           'AFYA', 'SMCI', 'HDSN', 'VIST', 'VRNA', 'NWG', 'PINE', 'VIR', 'ASO', 'PERI', 'STRL', 'VPG', \
           'AAPL', 'ABBV', 'AFL', 'ADBE', 'AIG', 'AMD', 'AMZN', 'AOS', 'ARNC', 'AVB', 'AVGO', \
           'AZO', 'BA', 'BAC', 'BBBY', 'BIIB', 'BNTX', 'CAG', 'CAT', 'CF', 'CL', \
           'CLX', 'CMCSA', 'CRM', 'CRWD', 'CSCO', 'CSX', 'CVS', 'CVX', 'DAC', 'DDOG', \
           'DHI', 'DUK', 'DVN', 'EPAM', 'EPD', 'ET', 'EXPE', 'FAF', 'FANG', 'FDX', 'FLNG', \
           'FMS', 'FNF', 'FSLY', 'FSR', 'FYBR', 'GD', 'GIS', 'GLNG', 'GOOGL', 'GS', 'GSL', 'HD', 'HON', \
           'IBM', 'INTC', 'IPI', 'IRM', 'JNJ', 'JPM', 'K', 'KO', 'LMT', 'LOW', \
           'LRCX', 'LYB', 'MA', 'MCK', 'META', 'MGA', 'MLM', 'MMM', 'MMP', 'MOS', 'MPW', \
           'MRK', 'MRNA', 'MRVL', 'MSFT', 'MU', 'NFLX', 'NKTR', 'NOW', 'NRG', 'NVDA', \
           'OKTA', 'ON', 'PANW', 'PFE', 'PHM', 'PII', 'PLTR', 'PRU', 'PYPL', 'QCOM', 'RITM', \
           'RIVN', 'ROST', 'RTX', 'SBLK', 'SBUX', 'SCHW', 'STOR', 'SKT', 'SNOW', 'SO', 'SPG', 'SPY', \
           'STWD', 'TSLA', 'TRV', 'TXN', 'TSCO', 'TSN', 'UNP', 'UPS', 'USAC', 'V', 'VICI', 'VGT', 'VMW', \
           'VNOM', 'VOO', 'WEC', 'WMT', 'WSM', 'WFC', 'XOM', 'ZIM', 'ZS')

HIDE_TICKERS = ('AOS', 'FSLY', 'NOW', 'PLTR', 'TSLA', 'VNOM', 'RIVN', 'NKTR') # Keep these tickers for database, but do not show them in best_options, because you do not want to own them.  

FILE_ID_TAG       = 'Tonka'

# *****************
#
# Import Libraries
#
# *****************

import csv
import copy
import json
import numpy as np
import os
import pandas as pd
import requests   # for http requests
import scipy
from   scipy import stats
from   scipy.stats import norm
import time
import yfinance as yf

import datetime
from   datetime import datetime
from   datetime import date
from   datetime import timedelta
#from   datetime import fromtimestamp

from   dateutil.relativedelta  import relativedelta
from   yahoo_earnings_calendar import YahooEarningsCalendar

#import investpy
#import glob
#import collections
#from enum import Enum
#import re     # re is regular expression. For searching strings.


# *****************
#
# Flow Control
# Internal Constants
#
# *****************

update_root_data      = True  #skip WTI call if testing code. Just use synthetic WTI_dict instead.
update_root_data_ER   = True  #skip call to yahoo calendar if testing code.
update_option_data    = True  #skip calls to yfinance if testing code.

profile = 'POWARRMax'

DATE_FORMAT       = '%Y-%m-%d %H:%M:%S'

TICKER_DELAY      = 0.020  # seconds between tickers for data requests.
EXPIRY_DELAY      = 0.005

MAX_DAYSOUT = 366.0  # This constant controls how many expiry strings are downloaded. Use low # to exclude BTOS and reduce ext. calls. Hi # includes BTOS.
MIN_DAYSOUT = 3.0

FEE_SPREAD = 0.75     # Fee in the bid/ask range.

# POWARRMax Standard Column Names. PAY ATTENTION!
# DO NOT CHANGE column names in all_options. Keep consistent with Yahoo Finance and my historical data. 
# Start column names below with best_STO. 

# Changed some column names to Yahoo Finance conventions. Think about this.

TRADE_LOG_COLS_DICT = {'TRADE_LOG_OPEN_COLS': ('Acct', 'Open Daysout', 'Expiry Date', 'Open Commsn', 'Open Date', \
                                               'Contracts', 'fee', 'root price', 'Option Type', \
                                               'root', 'Strike Price', 'Open Xtn Type', 'Open Net Proceeds'), \
                       'TRADE_LOG_OPEN_METRICS_COLS': ('impliedVolatility', 'Open POW', 'Open ARR', 'Open % Fee', 'Open % OTM', \
                                                       'Cash at Risk', 'Stock $ at Risk', 'Expiry Daysout'), \
                       'TRADE_LOG_CUR_COLS': ('Cur Price % Chg', 'Cur Root Price', 'Cur ARRBE Fee', 'Cur % OTM', \
                                              'Cur Fee / Contract', 'Cur IV', 'Cur Daysout'), \
                       'TRADE_LOG_CLOSE_COLS': ('Close Fee / Contract', 'Close Date', 'Close Root Price', 'Close IV', \
                                                'Close Net Proceeds', 'Close Daysout', 'Close Commsn', 'Close Xtn Type', \
                                                'Close ARR'), \
                       'HOLDINGS_COLS': ('Root', 'Shares', 'Committed Shares', 'Share Price', 'Total Value', \
                                         'Share Cost Basis')}
                                          
call_ticker_options = pd.DataFrame()
put_ticker_options  = pd.DataFrame()
ticker_options      = pd.DataFrame() # for a single ticker
all_options         = pd.DataFrame() # includes options for all tickers. Do not use any limits or exclusions for all_options.
good_options        = pd.DataFrame() # Has rankings for CSTO options and BTOs (CBTO).
stats               = pd.DataFrame() # Not used now.
options_open        = pd.DataFrame() # persistent file used by OptionCloser to recommend BTC offers. Dup of trade_log_open?
trade_log_open      = pd.DataFrame() 
    


In [2]:
# *****************
#
# Functions to convert broker reports into standard POWARRMax report formats for holdings and xtns.
# 
# *****************

def BrokerConverterVan(root_data, broker_report, TRADE_LOG_COLS_DICT):

    # *****************
    # NOTE - This version works with the Vanguard report, which awkwardly combines holdings and transactions
    #       into a single csv file. 
    #
    # 1. Read broker report with holdings and xtns, which Vanguard combines into a single df. 
    # 2. Parse report into three dfs: stock holdings, xtns, and open option holdings (trades). 
    #     For open_options parse option 'contractSymbol' into root, option type (P or C), strike price, and expiry date.
    #
    #
    # *****************
    
    # Vanguard Column Names
    
    VGD_COLS = ('Trade Date', 'Commission Fees', 'Share Price', 'Shares', 'Account Number', \
                'Accrued Interest', 'Account Type', 'Settlement Date', 'Investment Name', \
                'Transaction Description', 'Transaction Type', 'Net Amount', 'Principal Amount', \
                'Symbol')
    
    DROP_VGD_COLS = ['Account Number', 'Settlement Date', 'Transaction Description', 'Investment Name', 'Account Type']

    VAN_HOLDINGS_COLS = ('Account Number', 'Investment Name', 'Symbol', 'Shares', 'Share Price')
    
    OPTION_XTN_TYPES = ('Buy to open', 'Sell to open', 'Buy to close', 'Sell to close', 'Assignment', 'Expired')
    OPEN_XTN_TYPES   = ('Sell to open', 'Buy to open')
    CLOSING_XTN_TYPES = ('Sell to close', 'Buy to close', 'Assignment', 'Expired')

    # **********
    # From brokerage holdings and activity statement, create 3 separate dfs: -- holdings, opt_xtns, and open_options.
    
    # 1. Create holdings df from top rows of broker_report.
    xtns_row = broker_report.loc[broker_report['colx1'] == 'Trade Date']
    split_index = xtns_row.index.values.astype(int)[0] 
    holdings = broker_report[broker_report.index < split_index].copy()
   
    # Remove unused columns from holdings
    unused_cols = []
    for i in range(5, 15):
        col_name = 'colx' + str(i)
        unused_cols.append(col_name)
    
    holdings.drop(columns = unused_cols, inplace = True)
    
    # restore oroginal column headers, from row 1 and delete 'colx' headers
    holdings.rename(columns = holdings.iloc[0], inplace = True)
    holdings.drop(index = 0, inplace = True)
    holdings.drop(columns = 'Account Number', inplace = True)
    
    # 2. Create opt_xtns df from brokerage report. 
    #
    xtns = broker_report[broker_report.index >= split_index].copy()

    # Rename columns in xtns and reset index.
    xtns.columns = xtns.iloc[0]
    xtns = xtns[1:]
    xtns.reset_index(drop = True, inplace = True)
  
    # Select option trades only.
    opt_xtns = xtns[xtns['Transaction Type'].isin(OPTION_XTN_TYPES)].copy()
        
    opt_xtns.drop(DROP_VGD_COLS, axis = 1, inplace=True)
    opt_xtns.reset_index(drop = True, inplace = True)
        
    # 3. Create open_options df. This contains all the option trades open now, both puts and calls.
    #
    open_options = pd.DataFrame()
    open_options = holdings['Symbol'].str.split(' ',expand=True)
    open_options.rename(columns = {0:'Root', 1: 'Expiry Date', 2: 'Option Type', 3: 'Strike Price'}, inplace = True)
    open_options = pd.concat([holdings, open_options], axis = 1).copy()

    open_options = open_options[open_options['Option Type'].isin(['P', 'C'])].copy()
    open_options['Contracts'] = open_options['Shares']
    open_options['Contracts'] = open_options['Contracts'].astype(int)
    
    open_options.drop(columns = ['Investment Name', 'Symbol', 'Shares'], inplace = True)
    
#    Update 'Cur Root Price' column in open_options from price in root_data. 
#    print(root_data.head(10))
    open_options['Cur Root Price'] = 0.0
    unique_tickers = open_options['Root'].unique()
    for tick in unique_tickers:
        tick_indexes = open_options[open_options['Root'] == tick].index
        
    return holdings, open_options, opt_xtns


In [3]:

def BrokerConverterTDA(root_data, broker_option_holdings, broker_option_xtns, TRADE_LOG_COLS_DICT):

    # *****************
    # 1. Read broker report with option holdings.  
    # 2. For open_options parse option 'contractSymbol' into root, option type (P or C), strike price, and expiry date.
    # 3. Calculate 3 dfs for at risk dashbaord 
    #
    # *****************

    # TDA Column Names
    TDA_HOLDINGS_COLS = ('Symbol', 'Qty')
    TDA_XTN_COLS      = ('DATE', 'TRANSACTION ID', 'DESCRIPTION', 'QUANTITY', 'SYMBOL', 'PRICE', 'COMMISSION', 'AMOUNT')
    TDA_DROP_COLS     = ('REG FEE', 'SHORT-TERM RDM FEE', 'FUND REDEMPTION FEE', ' DEFERRED SALES CHARGE')
    
    OPTION_XTN_TYPES = ('Buy to open', 'Sell to open', 'Buy to close', 'Sell to close', 'Assignment', 'Expired')
    OPEN_XTN_TYPES   = ('Sell to open', 'Buy to open')
    CLOSING_XTN_TYPES = ('Sell to close', 'Buy to close', 'Assignment', 'Expired')

    
    # **********
    # From brokerage holdings and activity statements, create open_options.
    
    # Create holdings df from top rows of broker_report. 
    
    broker_option_holdings_mask = holdings_row_mask = broker_option_holdings.iloc[:,0] == 'Options'
    holdings_row = broker_option_holdings[broker_option_holdings_mask]
    
    split_index = holdings_row.index.values.astype(int)[0] + 3
    holdings = broker_option_holdings[broker_option_holdings.index >= split_index].copy()
    
    # Set column headers to first row, then reset index.
    holdings.columns = holdings.iloc[0]
    holdings = holdings[1:]
    holdings.reset_index(drop = True, inplace = True)
    
    # Drop empty rows.
    holdings = holdings[holdings['Symbol'].notna()].copy()
    holdings.reset_index(drop = True, inplace = True)
    
    # Drop last row.
    last_row = holdings.shape[0] - 1
    holdings.drop(index = last_row, inplace = True)
    
    # Create open_options df. This contains all the option trades open now, both puts and calls.
    #
    open_options = pd.DataFrame()
    open_options = holdings['Symbol'].str.split(' ',expand=True)
    open_options.rename(columns = {0:'Root', 1: 'month', 2: 'day', 3: 'year', 4: 'Strike Price', 5: 'Option Type', \
                                   6: 'Freq'}, inplace = True)
    open_options = pd.concat([holdings, open_options], axis = 1).copy()
    
    # Write expiry Date as datetime and string
    open_options['month'] = pd.to_datetime(open_options.month, format='%b').dt.month.astype(str)
    open_options['Expiry Date str'] = open_options['year'] +'-'+ open_options['month'] +'-'+ open_options['day']
    open_options['Expiry Date'] = pd.to_datetime(open_options['Expiry Date str'], infer_datetime_format = True)
    
    open_options['Contracts'] = open_options['Qty']
    open_options['Contracts'] = open_options['Contracts'].astype(int)
    
    # Convert all column names to PWARRMax standard.
    # For 'Option_Type' POWARRMax uses 'Put' and 'Call'
    
    #    TRADE_LOG_OPEN_COLS = ('Acct', 'Open Daysout', 'Expiry Date', 'Expiry Daysout', 'Open Action', 'Open Commsn' \
    #                       'Open Date', 'Contracts', 'Open Fee / Contract', 'Open Root Price', 'Option Type', \
    #                       'Root', 'Strike Price', 'Open Xtn Type')

    open_options.drop(columns = ['Underlying symbol', 'Symbol', 'Qty', 'month', 'day', 'year', \
                                 'Gain ($)', 'Maint req', 'Freq', 'Expiry Date str'], inplace = True)

    print(open_options.columns.tolist)
    print()
    
    # Update 'Cur Root Price' column in open_options from price in root_data. 
    open_options['Cur Root Price'] = 0.0
    print('after col. drop open options shape = ')
    print(open_options)
    
    unique_tickers = open_options['Root'].unique()
    for tick in unique_tickers:
        tick_indexes = open_options[open_options['Root'] == tick].index

    opt_xtns = pd.DataFrame()  #Need to create this eventually.
    
    return holdings, open_options, opt_xtns


In [4]:
#************
# Main Program - calls Broker_Converter
#************

# Main program tests Broker_Converter ONLY. It uses three test files: trade_log_open, trade_log_closed,
#      and opt_xtns. It reads these directly. 
root_data        = pd.read_csv('root_data.csv')
root_data.sort_values('Ticker', axis = 0, inplace = True, ignore_index = True)
root_data.set_index('Ticker', inplace = True)

best_STO = pd.read_csv('best_STO_Tonka.csv', index_col = None)
print(best_STO.head(10))

BROKERAGE = 'Vanguard' 

if BROKERAGE == 'TDA':
    broker_open_options = pd.read_csv('TDA_positions.csv')
    broker_xtns         = pd.read_csv('TDA_xtns.csv')
    holdings, open_options, opt_xtns = BrokerConverterTDA(root_data, broker_open_options, broker_xtns, TRADE_LOG_COLS_DICT)

if BROKERAGE == 'Vanguard':
    # Create temporary column headersbecause Vanguard concatenates the positions report and the transactions report.
    col_headers = []
    for i in range(15):
        col_name = 'colx' +str(i)
        col_headers.append(col_name)
        
    broker_report_van = pd.read_csv('broker_report_van.csv',header = None, names = col_headers, skip_blank_lines = True)
    
    holdings, open_options, opt_xtns = BrokerConverterVan(root_data, broker_report_van, TRADE_LOG_COLS_DICT)
    
if BROKERAGE == 'Skip':  # Use for testing. Skip parsing brokerage reports and use files in the repo.
    holdings      = pd.read_csv('holdings.csv')
    open_options = pd.read_csv('open_options.csv')
    opt_xtns     = pd.read_csv('opt_xtns.csv')
    
    print(' Skipped broker converters. Read files already in the repo.')
    
else:   # Write updated files to the repo.
    holdings.to_csv('holdings.csv', index = False)
    open_options.to_csv('open_options.csv', index = False)
    opt_xtns.to_csv('opt_xtns.csv', index = False)

    print('wrote 3 results files for Broker_Converter.')


   strike  impliedVolatility option_type  root  root price           ER Date  \
0    95.0                 48        call  ACLS       85.41               NaN   
1   370.0                 44        call  ADBE      340.30  2022-12-15 13:00   
2   370.0                 42        call  ADBE      340.30  2022-12-15 13:00   
3   360.0                 50        call  ADBE      340.30  2022-12-15 13:00   
4   365.0                 45        call  ADBE      340.30  2022-12-15 13:00   
5   360.0                 45        call  ADBE      340.30  2022-12-15 13:00   
6   365.0                 42        call  ADBE      340.30  2022-12-15 13:00   
7   310.0                 44         put  ADBE      340.30  2022-12-15 13:00   
8   315.0                 46         put  ADBE      340.30  2022-12-15 13:00   
9   320.0                 47         put  ADBE      340.30  2022-12-15 13:00   

       Expiry   fee  daysout    OTM  ARR  PctOTM  POW  PctFee  
0  2023-01-20  2.02       36   9.59   23   11.23   75  

In [5]:
#**********
#
# Little_Guns SetupFile Setup for little option guns.
# 1. Read best_options_suffix into best_options df for input to STO Guns. 
# 2. Create trade_log_open. Calc all addl columns. 
#      This code block goes back into OPtionGun when tested.

# Get option files for STO guns.
all_options = pd.read_csv('all_options_Tonka.csv', index_col = False)
best_options = pd.read_csv('best_STO_Tonka.csv', index_col = False)

# **********
# UpdateCurValues - Calculates current values for open trades based on curent share price and option price.
#                   This fct does not use current value of the option fee or current IV, but it may in the future.
# 1. Update root price.
# 2. Calculate all TRADE_LOG_CUR_COLS values. 
#
# **********      

def UpdateCurValues(root_data, open_trades, TRADE_LOG_COLS_DICT):
    # Update root price in open_trades. Get unique list of roots in open_trades, then for each ticker, update with a mask.
    
    # Update root prices in open_trades
    TICKERS = open_trades['Root'].unique().tolist()
    for ticker in TICKERS:
        open_trades['Cur Root Price'].mask(open_trades['Root'] == ticker, \
                                            other = root_data.loc['Ticker'] == ticker, 'root price'], inplace = True)
    # open_trades['Cur Price % Chg'] = 
        
    open_trades['Cur Price % Chg'] = (open_trades['Cur Root Price'] - open_trades['Open Root Price']) * 100 / \\
                                      open_trades['Open Root Price']
    open_trades['Today'] = pd.timestamp(today())
    open_trades['Cur Daysout'] = (open_trades['Expiry'] - open_trades['Today']) / np.timedelta64(1,'D')
    open_trades['Cur % OTM'] = (open_trades['Cur Root Price'] - open_trades['Strike']) * 100.0 / open_trades['Cur Root Price']
    open_trades['ARRBE Fee'] = 
    
    
        
    
# Add trade_log column names to xtns
TRADE_LOG_COLS_DICT = {'TRADE_LOG_OPEN_COLS': ('Acct', 'Open Daysout', 'Expiry Date', 'Open Commsn', 'Open Date', \
                                               'Contracts', 'Open Fee / Contract', 'Open Root Price', 'Option Type', \
                                               'Root', 'Strike Price', 'Open Xtn Type', 'Open Net Proceeds'), \
                       'TRADE_LOG_OPEN_METRICS_COLS': ('Open IV', 'Open POW', 'Open ARR', 'Open % Fee', 'Open % OTM', \
                                                       'Cash at Risk', 'Stock $ at Risk', 'Expiry Daysout'), \
                       'TRADE_LOG_CUR_COLS': ('Cur Price % Chg', 'Cur Root Price', 'Cur ARRBE Fee', 'Cur % OTM', \
                                              'Cur Fee / Contract', 'Cur IV', 'Cur Daysout'), \
                       'TRADE_LOG_CLOSE_COLS': ('Close Fee / Contract', 'Close Date', 'Close Root Price', 'Close IV', \
                                                'Close Net Proceeds', 'Close Daysout', 'Close Commsn', 'Close Xtn Type', \
                                                'Close ARR'), \
                       'HOLDINGS_COLS': ('Root', 'Shares', 'Committed Shares', 'Share Price', 'Total Value', \
                                         'Share Cost Basis')}
    
TL_OPEN_COLS         = TRADE_LOG_COLS_DICT['TRADE_LOG_OPEN_COLS']
TL_OPEN_METRICS_COLS = TRADE_LOG_COLS_DICT['TRADE_LOG_OPEN_METRICS_COLS']          
TL_CUR_COLS          = TRADE_LOG_COLS_DICT['TRADE_LOG_CUR_COLS']
    
# Calculate metrics columns for open_options.
# 


SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' on line 26 (2654423842.py, line 27)

In [ ]:
#**********
#
# Put_STO_Gun_01
# Started 12/14/22
# 1. Develop concentrations list from open_options.
# 2. Define max_concentration rule.
# 3. Define max_cash_total and max_cash_daily rules.
# 4. Define good_company rule.
# 5. Define priority for ranking best_options.
#
#**********

def Put_STO_Gun(holdings, best_options):
    PSTO_options = best_options[best_options['option_type'] == 'put'].copy()
    return PSTO_options

Run_PSTO = True

if Run_PSTO:
    PSTO_options = Put_STO_Gun(holdings, best_STO)
    write_file = 'Put_STO_' + FILE_ID_TAG + '.csv'
    PSTO_options.to_csv(write_file)
    
    print('ran Put_STO_Gun')
    print(PSTO_options.head(10))

In [ ]:
#**********
#
# Call_STO_Gun_01
# Started 12/14/22
# 1. From holdings, create list of holdings with > 100 shares.
# 2. Calculate # contracts possible
# 3. Subtract all contracts already used in open_options
# 4. Screen best_options for only the roots with contracts available
#
#**********

def Call_STO_Gun(holdings, best_options):
    CSTO_options = best_options[best_options['option_type'] == 'call'].copy()
    
    return CSTO_options

Run_CSTO = True
if Run_CSTO:
    CSTO_options = Call_STO_Gun(holdings, best_STO)
    write_file = 'Call_STO_' + FILE_ID_TAG + '.csv'
    CSTO_options.to_csv(write_file)
    
    print('ran Call_STO_Gun.')
    print(CSTO_options.head(10))
    

In [ ]:
#**********
#
# Put_BTC_Gun_01
# Started 12/14/22
# 1. From open_options create list of open puts
# 2. Screen all_options for only each open put.
# 2.1 Find matching PBTC trade. From all_options, get last price  
# 3. Annotate the put record with ARRBE price, < 20% price, etc. 
# 4. Make recommendation - Wait, place BTC for day / GTC, at price X, Wait for price drop to close, 
#        Wait for Expiry, Wait for Assignment.
#
# Make close offer IF:
#   - matching_PBTC_fee < MIN_BID_Pct or > ARRBE (with factor)  # for profitable trades
#           or concentration too high, or ITM is low enough, or cost_basis is higher than strike, or don't want the root.               
#**********

def Put_BTC_Gun(holdings, open_options, all_options, user_option_profile):
    # Select only Put open trades
    PBTC_trades = open_options[open_options['Option Type'] == 'put'].copy()
    PBTC_trades.reset_index(drop = True, inplace = True)
    PBTC_trades['row_num'] = PBTC_trades.index
    
    TRADE_LOG_COLS_DICT = {'TRADE_LOG_OPEN_COLS': ('Acct', 'Open Daysout', 'Expiry Date', 'Open Commsn', 'Open Date', \
                                                   'Contracts', 'Open Fee / Contract', 'Open Root Price', 'Option Type', \
                                                   'Root', 'Strike Price', 'Open Xtn Type', 'Open Net Proceeds'), \
                       'TRADE_LOG_OPEN_METRICS_COLS': ('Open IV', 'Open POW', 'Open ARR', 'Open % Fee', 'Open % OTM'), \
                                                       'Cash at Risk', 'Stock $ at Risk', 'Expiry Daysout'), \
                       'TRADE_LOG_CUR_COLS': ('Cur Price % Chg', 'Cur Root Price', 'Cur ARRBE Fee', 'Strike v Cur Root Price', \
                                              'Cur Fee / Contract', 'Cur Date', 'Cur Root Price', 'Cur IV', \
                                              'Cur Net Proceeds', 'Cur Daysout', 'Close Commsn'), \
                       'TRADE_LOG_CLOSE_COLS': ('Close Fee / Contract', 'Close Date', 'Close Root Price', 'Close IV', \
                                                'Close Net Proceeds', 'Close Daysout', 'Close Commsn', 'Close Xtn Type'), \
                                                'Close ARR'), \
                       'HOLDINGS_COLS': ('Root', 'Shares', 'Committed Shares', 'Share Price', 'Total Value', \
                                         'Expiry Date', 'Contracts', 'Option Type', 'Open Action', 'Strike Price', \
                                         'Contracts', 'Cur Fee / Contract')}

    matching_PBTC = pd.DataFrame()
    #Find matching PBTC trade in all_options
    if PBTC_trades.shape[0] > 0:
        for i in range(PBTC_trades.shape[0]):
            strike      = PBTC_trades.loc['row_num' == i, 'Strike Price']
            expiry      = PBTC_trades.loc['row_num' == i, 'Expiry Date']
            root        = PBTC_trades.loc['row_num' == i, 'Root']
            option_type = PBTC_trades.loc['row_num' == i, 'Option Type']
            matching_PBTC = all_options.mask((all_options['Strike Price'] == strike) & \
                                         (all_options['Expiry Date'] == expiry)) & \
                                        ((all_options['Option Type'] == option_type) & (all_options['Root'] == root))
            print('for PBTC_trades[, i, '], '] )
        
        # Get last price, IV, close from matching PBTC
        PBTC_trades.loc['row_num' == i, 'Close IV'] = matching_PBTC.loc[0, 'Implied Volatility']
        PBTC_trades.loc['row_num' == i, 'close_last price'] = matching_PBTC.loc[0, 'last price']
        PBTC_trades.loc['row_num' == i, 'bid_fee'] = matching_PBTC.loc[0, 'fee']
    
    #Calculate vector column values for PBTC_trades columns 
    PBTC_trades['ARRBE Fee'] = PBTC_trades['ARR'] * PBTC_trades['Cur Daysout'] * PBTC_trades['Strike'] / 365.0
    PBTC_trades['min_close_bid'] = user_option_profile.loc['BTC_close_Pct', 'value'] * PBTC_trades['fee']
    PBTC_trades['est_close_ARR'] = user_option_profile.loc['BTC_ARRBE_Pct', 'value'] * PBTC_trades['ARRBE Fee']

    # Add explanation.
    
    return PBTC_options

Run_PBTC = True

if Run_PBTC:
    PBTC_options = Put_BTC_Gun(holdings, open_options, all_options)
    write_file = 'Put_BTC_' + FILE_ID_TAG + '.csv'
    PBTC_options.to_csv(write_file)
    
    print('ran Put_BTC_Gun')

In [ ]:
#**********
#
# Call_BTC_Gun_01
# Started 12/14/22
# 1. From open_options create list of open_calls
# 2. Screen all_options for each of the open_calls
# 3. Annotate the call with ARRBE price, < 20% price, etc. 
#
#**********

def Call_BTC_Gun(holdings, all_options):
    
    
    return CBTC_options

Run_CBTC = True

if Run_CBTC:
    CBTC_options = Call_BTC_Gun(holdings, all_options)
    write_file = 'Call_BTC_' + FILE_ID_TAG + '.csv'
    CBTC_options.to_csv(write_file)
    
    print('ran Call_BTC_Gun.')
    
print('shot all four little guns.')
print()


In [ ]:
# ****************
#
#Calculate metrics for each option.
#
# *****************
all_options['Quote_Time'] = pd.to_datetime(all_options['Quote_Time'], format = DATE_FORMAT)
all_options['Expiry']     = pd.to_datetime(all_options['Expiry'], format = DATE_FORMAT)
all_options['fee']        = FEE_SPREAD * (all_options['ask'] - all_options['bid']) + all_options['bid']
all_options['daysout']    = (all_options['Expiry'] - all_options['Quote_Time']) / np.timedelta64(1,'D')
all_options['strike']     = pd.to_numeric(all_options['strike'], errors = 'coerce')
all_options['root price'] = pd.to_numeric(all_options['root price'], errors = 'coerce')
#all_options['ER Date']    = pd.to_datetime(all_options['ER Date'], format = DATE_FORMAT)
#all_options['ER daysout'] = (all_options['ER Date'] - all_options['Quote_Time']) / np.timedelta64(1,'D')
#all_options['ER-Expiry']  = all_options['ER daysout'] - all_options['daysout']

all_options['OTM']     = np.where(all_options['option_type'] == 'call', all_options['strike'] - all_options['root price'], \
                                  all_options['root price'] - all_options['strike'])

# use np.where to calculate different ARRs based on put/call and buy/sell
all_options['ARR']     = 100.0 * (all_options['fee'] * 365) / (all_options['daysout'] * all_options['root price'])
all_options['PctOTM']  = 100.0 * all_options['OTM'] / all_options['root price']

#all_options['TQI']     = all_options['ARR'] * all_options['PctOTM'] /10

all_options['callPITM'] = 100.0 * norm.cdf(np.log(all_options['root price'] / all_options['strike']) /
                                  (all_options['impliedVolatility'] * (all_options['daysout'] / 365)**(1/2)))

all_options['POW'] = np.where(all_options['option_type'] == 'call', 100.0 - all_options['callPITM'], all_options['callPITM'])

all_options['PctFee'] = 100.0 * all_options['fee'] / all_options['root price']
all_options['BidAskSpread'] = 100.0 * (all_options['ask'] - all_options['bid']) / all_options['bid']
all_options['impliedVolatility'] = 100.0 * all_options['impliedVolatility']

write_file = 'all_options' + FILE_ID_TAG + '.csv'
all_options.to_csv(write_file, index = False)
print('Wrote all_options')

